In [329]:
# utils
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import random
import unidecode

# image
from PIL import Image, ImageFilter

# openCV
import cv2

# scikit learn
from skimage import io, util
from skimage.color import rgb2gray

# imgaug
import imgaug as ia
from imgaug import augmenters as iaa

# noise
import noise

In [19]:
# loading test image
img = cv2.imread('test.jpg')

In [357]:
# draw a randomic overlay to be merged with image
def random_overlay(shape, color=None):
    import random
    import cv2
    # get image shape
    height, width, channels = shape

    # get random color
    color_range = random.choice(range(0, 55))
    color = (color_range, color_range, color_range)
    
    # create blank white image
    overlay = 255 * np.ones(shape=[height, width, channels], dtype=np.uint8)
    
    # randomize crappy shape and position
    ## 0 - rectangle
    ## 1 - line
    ## 2 - circle
    for i in range(10):
        # get randomic shape position
        x1, y1 = [random.choice(range(0, width)), random.choice(range(0, height))]
        x2, y2 = [x1 + random.choice(range(0, 200)), y1 + random.choice(range(0, 200))]
        
        # select random shape
        shape_range = range(0, 2)
        randomic_shape = random.choice(shape_range)
        if randomic_shape == 0:
            cv2.rectangle(overlay, (x1, y1), (x2, y2), color, -1)
        elif randomic_shape == 1:
            random_thickness = random.choice(range(0, 80))
            cv2.line(overlay, (x1, y1), (x2, y2), color, random_thickness)        
        else:
            cv2.circle(overlay, (x1, y1), (x2, y2), color, -1)
    overlay = cv2.blur(overlay, (150, 150))
    return overlay

In [404]:
def crappy_pipeline(image):
    # image values
    height, width, channels = image.shape
    
    # apply some ImageFilter filters from PIL to add noise to borders
    pil_image = Image.fromarray(image)
    allowed_pil_filters = ['EDGE_ENHANCE_MORE', 'CONTOUR', 'SHARPEN']
    random_pil_filter = random.choice(allowed_pil_filters)
    pil_image = pil_image.filter(ImageFilter.__dict__[random_pil_filter])
    
    # draw random shapes on image using OpenCV
    opencv_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    overlay = random_overlay(opencv_image.shape)
    alpha = 0.6
    opencv_image = cv2.addWeighted(opencv_image, alpha, overlay, 1 - alpha, 0)
    
    # add some imgaug augmentation methods to add noise to image
    ## define function to randomize choice of a augment execution
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    
    sequential_aug_pipeline = iaa.Sequential([
        iaa.PerspectiveTransform(scale=0.025),
        sometimes(
            iaa.SaltAndPepper(p=0.05)
        ),
        iaa.SomeOf((0, 5), [
#             iaa.CoarseSaltAndPepper(p=0.01,size_percent=0.9),
            iaa.GammaContrast(5),
            iaa.JpegCompression(compression=10)
        ]),
        iaa.Dropout(p=0.001),
        iaa.Affine(rotate=0.05),
        sometimes(iaa.SimplexNoiseAlpha(
            iaa.EdgeDetect(0.1),
            upscale_method="linear"
        ))
    ])
    
    imgaug_image = sequential_aug_pipeline.augment_image(opencv_image)
    
    # noise lib to add perlin noise
    x1, y1 = [random.choice(range(0, width)), random.choice(range(0, height))]
    x2, y2 = [x1 + random.choice(range(0, 200)), y1 + random.choice(range(0, 200))]
    if x2 > width:
        x2 = width
    
    if y2 > height:
        y2 = height
    scale = 100.0
    octaves = 3
    persistence = 0.5
    lacunarity = 2.0
    
    noise_image = imgaug_image
    print(noise_image.shape)
    print(x1, x2, y1, y2)
    for i in range(x1, x2):
        for j in range(y1, y2):
#             noise_image[j][i] = noise_image[j][i] * 0.5
            noise_image[j][i] = noise.pnoise2(
                    i/scale, 
                    j/scale, 
                    octaves=octaves, 
                    persistence=persistence, 
                    lacunarity=lacunarity, 
                    repeatx=x2 - x1, 
                    repeaty=y2 - y1, 
                    base=1
                )
    Image.fromarray(noise_image).show()
    return noise_image

In [405]:
crappy_image = crappy_pipeline(img)

(1131, 800, 3)
369 431 413 429
